In [ ]:
# Standard libraries
import pickle  # For loading serialized Python objects
import numpy as np  # Numerical computations
import pandas as pd  # Data manipulation
import matplotlib.pyplot as plt  # Plotting
import seaborn as sns  # Statistical data visualization
import warnings  # For managing warnings

# Scikit-learn utilities
from sklearn.model_selection import GroupKFold, GridSearchCV  # Cross-validation and hyperparameter tuning
from sklearn.preprocessing import StandardScaler  # Feature scaling
from sklearn.impute import SimpleImputer  # Missing value imputation
from sklearn.feature_selection import SelectKBest, f_classif  # Feature selection using ANOVA F-test
from sklearn.metrics import (
    classification_report, accuracy_score, confusion_matrix, roc_curve, auc,
    f1_score, precision_score, recall_score, hamming_loss, roc_auc_score
)
from sklearn.base import BaseEstimator, TransformerMixin  # Custom transformers
from sklearn.utils import shuffle, compute_class_weight  # Utilities

# Imbalanced learning
from imblearn.over_sampling import SMOTENC  # SMOTE for mixed data types
from imblearn.under_sampling import RandomUnderSampler  # Random undersampling

# Model and explainability
from catboost import CatBoostClassifier, Pool  # Gradient boosting model
import shap  # SHAP values for model interpretation

# System utilities
import os  # Directory and path handling
import traceback  # Traceback printing for exception handling

# Suppress warnings
warnings.filterwarnings('ignore')

# === Custom transformer to drop columns with all NaNs ===
class DropAllNaNColumns(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        X_df = pd.DataFrame(X)
        self.non_nan_cols_ = ~np.all(X_df.isna(), axis=0)
        return self
    def transform(self, X):
        X_df = pd.DataFrame(X)
        return X_df.loc[:, self.non_nan_cols_].values

# === Load EEG and cohort data ===
file_path = "/Volumes/NOC_Drive/databases/biomarker_data.pkl"
with open(file_path, "rb") as file:
    hbn_data = pickle.load(file)

# Extract EEG feature data and cohort metadata
eeg_data = hbn_data["HBN_electrode_space"]["eeg_xdata"]
cohort_df = hbn_data["HBN_electrode_space"]["cohort_df"].copy()

# Extract subject IDs and condition (eyes open/closed) from filename
cohort_df['subject_uid'] = cohort_df['eeg_filename'].str.extract(r'HBN-sub-(NDAR[A-Z0-9]+)')[0]
cohort_df['condition'] = cohort_df['eeg_filename'].str.extract(r'_(EO|EC)_')[0]

# Identify binary disorder columns (only 2 unique values)
disorder_cols = [col for col in cohort_df.columns if cohort_df[col].nunique() == 2 and cohort_df[col].dtype in ['int64', 'float64']]
print("Disorders found:", disorder_cols)

# List of disorder pairs to compare
disorder_pairs = [
    ('ASD', 'ADHD-Combined Type'),
    ('Healthy controls', 'Anxiety Disorders'),
    ('SLD (Reading)', 'SLD (Mathematics)'),
    ('OCD', 'OCD-Other'),
    ('Communication Disorders', 'Intellectual Disability'),
    ('Depressive Disorders', 'Trauma and Stressor Related Disorders'),
    ('ADHD-Inattentive Type', 'ADHD-Hyperactive/Impulsive Type'),
    ('Oppositional Defiant Disorder', 'Disruptive, Impulse Control and Conduct Disorders-Other'),
    ('Tic Disorders', 'ASD'),
    ('ADHD-Combined Type', 'SLD (Written Expression)'),
    ('Anxiety Disorders', 'Depressive Disorders'),
    ('Elimination Disorders', 'OCD')
]

# Keep only those pairs where both disorders exist in the data
available_disorders = set(disorder_cols)
disorder_pairs = [(d1, d2) for d1, d2 in disorder_pairs if d1 in available_disorders and d2 in available_disorders]
print(f"Evaluating {len(disorder_pairs)} valid disorder pairs...")

# Create directories for results
output_dir = "./disorder_pair_results"
shap_dir = os.path.join(output_dir, "shap_values")
os.makedirs(output_dir, exist_ok=True)
os.makedirs(shap_dir, exist_ok=True)

# Mapping from subject-condition pair to index in EEG data
ids = eeg_data.coords['subject_uid'].values
conditions = eeg_data.coords['condition'].values
index_map = {(uid, cond): i for i, (uid, cond) in enumerate(zip(ids, conditions))}

# Helper function to map dataframe rows to EEG data indices
def map_indices(df_subset):
    indices = []
    for row in df_subset.itertuples():
        idx = index_map.get((row.subject_uid, row.condition))
        if idx is not None:
            indices.append(idx)
    return indices

# Initialize global tracking variables
all_true_labels = []
all_pred_labels = []
all_pred_probs = []
all_results = []

# === Main loop over disorder pairs ===
for i, (disorder1, disorder2) in enumerate(disorder_pairs, 1):
    print(f"\n========== Processing disorder pair {i}/{len(disorder_pairs)}: {disorder1} vs {disorder2} ==========")
    try:
        # Select only those rows with exclusive labels for each disorder
        df_binary = cohort_df[
            ((cohort_df[disorder1] == 1) & (cohort_df[disorder2] == 0)) |
            ((cohort_df[disorder1] == 0) & (cohort_df[disorder2] == 1))
        ].copy()

        # Skip if too few samples
        if df_binary.shape[0] < 10:
            print(f"Skipping pair {disorder1}-{disorder2}: Not enough samples ({df_binary.shape[0]})")
            continue

        # Label is 1 if disorder2, 0 if disorder1
        df_binary['label'] = df_binary[disorder2].values
        indices = map_indices(df_binary)

        # Skip if no corresponding EEG data found
        if len(indices) == 0:
            print(f"Skipping pair {disorder1}-{disorder2}: No matching EEG data indices found")
            continue

        # Subset EEG data
        eeg_selected = eeg_data.isel(ID=indices)
        n_samples, n_sources, n_freqs, n_biomarkers = eeg_selected.shape

        # Generate feature names
        eeg_feature_names_all = [
            f"biomarker_{b}_freq_{f}_source_{s}"
            for s in range(n_sources)
            for f in range(n_freqs)
            for b in range(n_biomarkers)
        ]

        # Reshape EEG tensor to 2D array
        X_raw = eeg_selected.values.transpose(0, 3, 2, 1).reshape(len(indices), -1)

        # Prepare feature dataframe
        condition_cat = df_binary['condition'].astype('category').reset_index(drop=True)
        X_raw_df = pd.DataFrame(X_raw)
        X_df = pd.concat([X_raw_df, condition_cat], axis=1)
        X_df.columns = list(map(str, range(X_raw.shape[1]))) + ['condition']
        y = df_binary['label'].values
        groups = df_binary['subject_uid'].values

        # Preprocessing
        dropper = DropAllNaNColumns()
        imputer = SimpleImputer(strategy='mean')
        scaler = StandardScaler()
        outer_cv = GroupKFold(n_splits=10)  # Ensures subject independence

        # Hyperparameter grid and metrics setup
        param_grid = {'depth': [4, 6], 'learning_rate': [0.01, 0.05], 'l2_leaf_reg': [3, 5]}
        cat_features = ['condition']
        roc_curves = []
        fold_metrics = {k: [] for k in ['fold', 'accuracy', 'f1_micro', 'f1_macro', 'precision_macro', 'recall_macro', 'hamming_loss', 'auc']}

        # === Cross-validation loop ===
        for fold, (train_idx, test_idx) in enumerate(outer_cv.split(X_df, y, groups=groups), 1):
            print(f"\n--- Outer Fold {fold} ---")

            # Train-test split
            X_train = X_df.iloc[train_idx].copy()
            y_train = y[train_idx]
            X_test = X_df.iloc[test_idx].copy()
            y_test = y[test_idx]

            # Separate and preprocess EEG data
            X_train_eeg = X_train.drop(columns=cat_features).values
            X_train_cat = X_train[cat_features].reset_index(drop=True)
            X_test_cat = X_test[cat_features].reset_index(drop=True)

            # Clean EEG features
            X_train_eeg = dropper.fit_transform(X_train_eeg)
            X_train_eeg = imputer.fit_transform(X_train_eeg)

            # Select top EEG features
            k_features = min(int(0.2 * X_train_eeg.shape[1]), 1000)
            selector = SelectKBest(score_func=f_classif, k=k_features)
            X_train_eeg = selector.fit_transform(X_train_eeg, y_train)
            selected_indices = selector.get_support(indices=True)
            selected_feature_names = [eeg_feature_names_all[i] for i in selected_indices]
            X_train_eeg = scaler.fit_transform(X_train_eeg)

            # Reconstruct training DataFrame for CatBoost
            X_train_bal_df = pd.DataFrame(X_train_eeg, columns=[f'feat_{i}' for i in range(X_train_eeg.shape[1])])
            X_train_bal_cat = pd.DataFrame(X_train_cat.values[np.random.choice(X_train_cat.shape[0], len(X_train_eeg), replace=True)],
                                           columns=X_train_cat.columns).reset_index(drop=True)
            X_train_bal_df = pd.concat([X_train_bal_df.reset_index(drop=True), X_train_bal_cat], axis=1)
            cat_idx = [X_train_bal_df.columns.get_loc(col) for col in cat_features]

            # Balance classes with SMOTENC and undersampling
            desired_ratio = 0.6
            n_majority = np.sum(y_train == 0)
            n_minority = np.sum(y_train == 1)
            target_minority = int(n_majority * desired_ratio)
            if n_minority < target_minority and n_minority > 1:
                smote_nc = SMOTENC(categorical_features=cat_idx, sampling_strategy={1: target_minority},
                                   random_state=fold, k_neighbors=min(5, n_minority - 1))
                X_train_bal_arr, y_train_bal = smote_nc.fit_resample(X_train_bal_df.values, y_train)
            else:
                X_train_bal_arr, y_train_bal = X_train_bal_df.values, y_train

            # Undersample majority class
            rus = RandomUnderSampler(sampling_strategy='auto', random_state=fold)
            X_train_bal_arr, y_train_bal = rus.fit_resample(X_train_bal_arr, y_train_bal)
            X_train_bal_df = pd.DataFrame(X_train_bal_arr, columns=X_train_bal_df.columns)

            # Process test set similarly
            X_test_eeg = X_test.drop(columns=cat_features).values
            X_test_eeg = dropper.transform(X_test_eeg)
            X_test_eeg = imputer.transform(X_test_eeg)
            X_test_eeg = selector.transform(X_test_eeg)
            X_test_eeg = scaler.transform(X_test_eeg)
            X_test_ready = pd.DataFrame(X_test_eeg, columns=[f'feat_{i}' for i in range(X_test_eeg.shape[1])])
            X_test_ready = pd.concat([X_test_ready.reset_index(drop=True), X_test_cat], axis=1)

            # Set categorical columns
            for df in [X_train_bal_df, X_test_ready]:
                for col in cat_features:
                    df[col] = df[col].astype('category')

            # Compute class weights for imbalance
            class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train_bal), y=y_train_bal)
            class_weights_dict = dict(zip(np.unique(y_train_bal), class_weights))

            # Train model with grid search
            base_clf = CatBoostClassifier(iterations=100, eval_metric='F1:use_weights=true',
                                          random_seed=42, verbose=0, early_stopping_rounds=30,
                                          class_weights=class_weights_dict)
            grid = GridSearchCV(base_clf, param_grid, cv=3, scoring='f1', n_jobs=-1)
            grid.fit(X_train_bal_df, y_train_bal, cat_features=cat_features)
            best_model = grid.best_estimator_

            # Evaluate on test set
            train_pool = Pool(X_train_bal_df, y_train_bal, cat_features=cat_features)
            test_pool = Pool(X_test_ready, y_test, cat_features=cat_features)
            best_model.fit(train_pool, eval_set=test_pool, use_best_model=True)
            y_proba = best_model.predict_proba(test_pool)[:, 1]
            fpr, tpr, thresholds = roc_curve(y_test, y_proba)
            roc_curves.append((fpr, tpr))
            optimal_threshold = thresholds[np.argmax(tpr - fpr)]
            y_pred = (y_proba >= optimal_threshold).astype(int)

            # Save metrics
            all_true_labels.extend(y_test)
            all_pred_labels.extend(y_pred)
            all_pred_probs.extend(y_proba)
            fold_metrics['fold'].append(fold)
            fold_metrics['accuracy'].append(accuracy_score(y_test, y_pred))
            fold_metrics['f1_micro'].append(f1_score(y_test, y_pred, average='micro'))
            fold_metrics['f1_macro'].append(f1_score(y_test, y_pred, average='macro'))
            fold_metrics['precision_macro'].append(precision_score(y_test, y_pred, average='macro', zero_division=0))
            fold_metrics['recall_macro'].append(recall_score(y_test, y_pred, average='macro', zero_division=0))
            fold_metrics['hamming_loss'].append(hamming_loss(y_test, y_pred))
            fold_metrics['auc'].append(auc(fpr, tpr))

            print(f"Fold {fold} metrics for {disorder1} vs {disorder2}:")
            print(f"  Accuracy: {fold_metrics['accuracy'][-1]:.4f}")
            print(f"  F1 Micro: {fold_metrics['f1_micro'][-1]:.4f}")
            print(f"  F1 Macro: {fold_metrics['f1_macro'][-1]:.4f}")
            print(f"  Precision Macro: {fold_metrics['precision_macro'][-1]:.4f}")
            print(f"  Recall Macro: {fold_metrics['recall_macro'][-1]:.4f}")
            print(f"  Hamming Loss: {fold_metrics['hamming_loss'][-1]:.4f}")
            print(f"  AUC: {fold_metrics['auc'][-1]:.4f}")

            # Save SHAP values for interpretability
            explainer = shap.TreeExplainer(best_model)
            shap_values = explainer.shap_values(train_pool)
            shap_df = pd.DataFrame(shap_values, columns=selected_feature_names + cat_features)
            shap_df['label'] = y_train_bal
            shap_df.to_csv(os.path.join(shap_dir, f"shap_{disorder1.replace(' ', '_')}_vs_{disorder2.replace(' ', '_')}_fold{fold}.csv"), index=False)

        # Aggregate and save metrics for this pair
        pair_metrics = {
            'disorder1': disorder1,
            'disorder2': disorder2,
            'accuracy_mean': np.mean(fold_metrics['accuracy']),
            'f1_micro_mean': np.mean(fold_metrics['f1_micro']),
            'f1_macro_mean': np.mean(fold_metrics['f1_macro']),
            'precision_macro_mean': np.mean(fold_metrics['precision_macro']),
            'recall_macro_mean': np.mean(fold_metrics['recall_macro']),
            'hamming_loss_mean': np.mean(fold_metrics['hamming_loss']),
            'auc_mean': np.mean(fold_metrics['auc'])
        }
        all_results.append(pair_metrics)
        pd.DataFrame([pair_metrics]).to_csv(f"{output_dir}/metrics_{disorder1}_vs_{disorder2}.csv", index=False)
        pd.DataFrame(fold_metrics).to_csv(f"{output_dir}/fold_metrics_{disorder1}_vs_{disorder2}.csv", index=False)

        # Plot fold metrics
        plt.figure(figsize=(12, 6))
        sns.lineplot(data=pd.DataFrame(fold_metrics).set_index('fold'))
        plt.title(f"Fold-wise Metrics for {disorder1} vs {disorder2}")
        plt.xlabel("Fold")
        plt.ylabel("Score")
        plt.tight_layout()
        plt.savefig(f"{output_dir}/fold_metrics_plot_{disorder1}_vs_{disorder2}.png")
        plt.close()

    except Exception as e:
        print(f"Error processing {disorder1} vs {disorder2}: {e}")
        traceback.print_exc()

# Save summary of all disorder pair metrics
summary_df = pd.DataFrame(all_results)
summary_df.to_csv(f"{output_dir}/all_disorder_pairs_metrics_summary.csv", index=False)

print("\nProcessing completed for all disorder pairs.")


In [ ]:
# Import necessary libraries
import numpy as np
import pandas as pd
import os
from collections import defaultdict

# Function to compute average SHAP importances per condition (disorder) in a binary classification pair
def average_shap_importances_per_disorder(pair_name, shap_dir):
    # Get list of all SHAP value files for the specified disorder pair
    fold_files = [f for f in os.listdir(shap_dir)
                  if f.startswith(f"shap_{pair_name}_fold") and f.endswith(".csv")]
    fold_files.sort()  # Ensure consistent ordering of folds

    # Initialize nested dictionary:
    # disorder_feature_shap[condition][feature] = list of SHAP values across samples
    disorder_feature_shap = defaultdict(lambda: defaultdict(list))

    # Iterate over each SHAP file (one per fold)
    for file in fold_files:
        shap_df = pd.read_csv(os.path.join(shap_dir, file))  # Load SHAP values
        feature_cols = [col for col in shap_df.columns if col not in ['condition', 'label']]  # Get feature columns

        # Iterate through each row (sample) in the SHAP dataframe
        for _, row in shap_df.iterrows():
            disorder = row['condition']  # Get the condition ('EO' or 'EC', or other categorical indicator)
            for col in feature_cols:
                try:
                    shap_val = abs(row[col])  # Use absolute SHAP value for importance
                    disorder_feature_shap[disorder][col].append(shap_val)
                except:
                    continue  # Skip any problematic columns (e.g., missing values)

    # Compute average SHAP value and rank features by importance for each condition
    disorder_avg_ranks = {}
    for disorder, feat_dict in disorder_feature_shap.items():
        # Calculate mean SHAP value for each feature
        avg_importance = {feat: np.mean(vals) for feat, vals in feat_dict.items()}
        # Sort features by average importance in descending order
        ranked_features = sorted(avg_importance.items(), key=lambda x: x[1], reverse=True)
        # Store the ranked features per disorder
        disorder_avg_ranks[disorder] = ranked_features

    return disorder_avg_ranks

# Example usage with a specific disorder pair
pair_name = "ADHD-Combined_Type_vs_SLD_(Written_Expression)"  # Filename-compatible version of disorder pair
shap_dir = "/Users/tuanadurmayuksel/Desktop/Internship_code/hbn-dataset-special-data/HBN /Final/montage/disorder_pair_results/shap_values/Shap"

# Run the function to get average feature importance per disorder
result = average_shap_importances_per_disorder(pair_name, shap_dir)

# Optional: Convert the result to a DataFrame for easier analysis or visualization
def result_to_df(disorder_avg_ranks):
    all_rows = []
    for disorder, feats in disorder_avg_ranks.items():
        for rank, (feat, avg_val) in enumerate(feats, 1):
            all_rows.append({'Feature': feat, 'Avg_SHAP': avg_val, 'Rank': rank})
    return pd.DataFrame(all_rows)

# Convert the SHAP results to a DataFrame
df_result = result_to_df(result)

# Display top 20 most important features across disorders
print(df_result.head(20))


In [ ]:
# Mapping lists
biomarkers = ['AbsolutePower', 'RelativePower', 'DFA', 'fEI']

frequencies = [
    '1.0-4.0 Hz', '4.0-5.1 Hz', '5.1-6.5 Hz', '6.5-8.3 Hz', '8.3-10.5 Hz',
    '10.5-13.4 Hz', '13.4-17.0 Hz', '17.0-21.7 Hz', '21.7-27.6 Hz',
    '27.6-35.2 Hz', '35.2-44.8 Hz'
]

sources = [
    'Cz','E1', 'E2', 'E3', 'E4', 'E5', 'E6', 'E7', 'E8', 'E9', 'E10', 'E11', 'E12', 'E13',
    'E14', 'E15', 'E16', 'E17', 'E18', 'E19', 'E20', 'E21', 'E22', 'E23', 'E24', 'E25',
    'E26', 'E27', 'E28', 'E29', 'E30', 'E31', 'E32', 'E33', 'E34', 'E35', 'E36', 'E37',
    'E38', 'E39', 'E40', 'E41', 'E42', 'E43', 'E44', 'E45', 'E46', 'E47', 'E48', 'E49',
    'E50', 'E51', 'E52', 'E53', 'E54', 'E55', 'E56', 'E57', 'E58', 'E59', 'E60', 'E61',
    'E62', 'E63', 'E64', 'E65', 'E66', 'E67', 'E68', 'E69', 'E70', 'E71', 'E72', 'E73',
    'E74', 'E75', 'E76', 'E77', 'E78', 'E79', 'E80', 'E81', 'E82', 'E83', 'E84', 'E85',
    'E86', 'E87', 'E88', 'E89', 'E90', 'E91', 'E92', 'E93', 'E94', 'E95', 'E96', 'E97',
    'E98', 'E99', 'E100', 'E101', 'E102', 'E103', 'E104', 'E105', 'E106', 'E107',
    'E108', 'E109', 'E110', 'E111', 'E112', 'E113', 'E114', 'E115', 'E116', 'E117',
    'E118', 'E119', 'E120', 'E121', 'E122', 'E123', 'E124', 'E125', 'E126', 'E127',
    'E128', 
]

# Function to parse and map a feature name
def parse_feature_name(feature):
    try:
        parts = feature.split('_')
        freq_idx = int(parts[1])
        biomarker_idx = int(parts[3])
        source_idx = int(parts[5])
        return {
            'Feature': feature,
            'Frequency': frequencies[freq_idx] if freq_idx < len(frequencies) else 'Unknown',
            'Biomarker': biomarkers[biomarker_idx] if biomarker_idx < len(biomarkers) else 'Unknown',
            'Source': sources[source_idx] if source_idx < len(sources) else 'Unknown'
        }
    except Exception as e:
        return {
            'Feature': feature,
            'Frequency': 'Error',
            'Biomarker': 'Error',
            'Source': 'Error'
        }

# Map features in result DataFrame
def map_feature_descriptions(df_result, top_n=20):
    mapped = [parse_feature_name(feat) for feat in df_result['Feature'].head(top_n)]
    df_mapped = pd.DataFrame(mapped)
    df_mapped['Avg_SHAP'] = df_result['Avg_SHAP'].head(top_n).values
    df_mapped['Rank'] = df_result['Rank'].head(top_n).values
    return df_mapped

# Example usage
df_mapped_result = map_feature_descriptions(df_result, top_n=20)
print(df_mapped_result)


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Load your combined metrics CSV
file_path = "/Users/tuanadurmayuksel/Desktop/Internship_code/hbn-dataset-special-data/HBN /Final/montage/disorder_pair_results/all_disorder_pairs_metrics_summary.csv"
df = pd.read_csv(file_path)


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

# Load data
file_path = "/Users/tuanadurmayuksel/Desktop/Internship_code/hbn-dataset-special-data/HBN /Final/montage/disorder_pair_results/all_disorder_pairs_metrics_summary.csv"
df = pd.read_csv(file_path)

# Disorder name mapping
disorder_name_map = {
    'ASD': 'ASD', 'ADHD-Combined Type': 'ADHD-C', 'Healthy controls': 'Healthy',
    'Anxiety Disorders': 'Anxiety', 'SLD (Reading)': 'SLD-Read', 'SLD (Mathematics)': 'SLD-Math',
    'SLD (Written Expression)': 'SLD-Write', 'OCD': 'OCD', 'OCD-Other': 'OCD-Other',
    'Communication Disorders': 'CommDis', 'Intellectual Disability': 'ID',
    'Depressive Disorders': 'Depression', 'Trauma and Stressor Related Disorders': 'Trauma-Stress',
    'Oppositional Defiant Disorder': 'ODD', 
    'Disruptive, Impulse Control and Conduct Disorders-Other': 'Impulse',
    'Tic Disorders': 'Tic', 'Elimination Disorders': 'Elimination',
    'ADHD-Hyperactive/Impulsive Type': 'ADHD-H', 'ADHD-Inattentive Type': 'ADHD-I'
}

# Create pair names
df['pair'] = df['disorder1'].map(disorder_name_map) + ' vs ' + df['disorder2'].map(disorder_name_map)

# Define metrics
metrics = ['accuracy_mean', 'f1_micro_mean', 'f1_macro_mean', 
           'precision_macro_mean', 'recall_macro_mean', 'auc_mean']
pretty_metrics = ['Accuracy', 'F1 Micro', 'F1 Macro', 'Precision', 'Recall', 'AUC']

# Prepare heatmap data
heatmap_data = df.set_index('pair')[metrics]
heatmap_data.columns = pretty_metrics
heatmap_data = heatmap_data.apply(lambda col: col.fillna(col.mean()), axis=0)

# Define desired order manually
desired_order = [
    "CommDis vs ID",
    "Elimination vs OCD",
    "ASD vs ADHD-C",
    "ODD vs Impulse",
    "Depression vs Trauma-Stress",
    "Anxiety vs Depression",
    "Tic vs ASD",
    "SLD-Read vs SLD-Math",
    "ADHD-C vs SLD-Write",
    "OCD vs OCD-Other",
    "Healthy vs Anxiety"
]

# Filter and reorder DataFrame
heatmap_data = heatmap_data.loc[desired_order]

# Numbered labels for legend
numbered_labels = [f"{i+1}. {pair}" for i, pair in enumerate(desired_order)]

# Replace index with numbers
heatmap_data.index = list(range(1, len(desired_order) + 1))

# Use a light blue gradient
light_green_cmap = sns.light_palette("#66BB6A", as_cmap=True)  # A soft pastel green


# Plot
fig, ax = plt.subplots(figsize=(14, 12))

sns.heatmap(
    heatmap_data,
    annot=True,
    fmt=".2f",
    cmap=light_green_cmap,
    annot_kws={"size": 30, "weight": "bold", "color": "black"},
    cbar_kws={'label': 'Metric Value'},
    ax=ax
)

# Axis label formatting
ax.set_xticklabels(ax.get_xticklabels(), rotation=15, fontsize=30, color='black')
ax.set_yticklabels(ax.get_yticklabels(), rotation=0, fontsize=30,  color='black')
ax.set_xlabel('Metrics', fontsize=30, fontweight='bold', color='black')
ax.set_ylabel('Disorder Pairs (Numbered)', fontsize=30, fontweight='bold', color='black')

# Colorbar formatting
cbar = ax.collections[0].colorbar
cbar.set_label('Metric Value', fontsize=30, color='black',fontweight='bold')
cbar.ax.tick_params(labelsize=30)

# Adjust layout to add label text at bottom
plt.subplots_adjust(bottom=0.35)

# Add disorder pair legend below plot
fig.text(0.5, 0.01, "\n".join(numbered_labels), ha='center', va='top', fontsize=16)

plt.tight_layout()
plt.show()




In [ ]:
# Select the metric columns to visualize
metrics_cols = [
    'accuracy_mean', 'f1_micro_mean', 'f1_macro_mean',
    'precision_macro_mean', 'recall_macro_mean', 'auc_mean'
]
# Friendly labels for the radar chart axes
pretty_labels = ['Accuracy', 'F1 Micro', 'F1 Macro', 'Precision', 'Recall', 'AUC']

# Extract the relevant data from your DataFrame
data = df[metrics_cols]

# Calculate the mean and standard deviation for each metric across folds/samples
avg_metrics = data.mean()
std_metrics = data.std()

# Prepare arrays for plotting
metrics = avg_metrics.values              # Mean values for each metric
stds = std_metrics.values                 # Standard deviations for each metric
labels = pretty_labels                    # Axis labels
num_vars = len(labels)                    # Number of axes

# Compute the angle for each axis (equal spacing around the circle)
angles = np.linspace(0, 2 * np.pi, num_vars, endpoint=False).tolist()
angles += angles[:1]  # Append first angle to close the loop

# Close the data loops for plotting
metrics = np.append(metrics, metrics[0])
stds = np.append(stds, stds[0])

# Calculate upper and lower bounds for the std deviation band
upper = metrics + stds
lower = metrics - stds

# Define benchmark cases for comparison on the radar chart
best_case = np.ones(num_vars + 1)        # Perfect case: all metrics = 1
worst_case = np.zeros(num_vars + 1)      # Worst case: all metrics = 0
avg_case = np.full(num_vars + 1, 0.5)    # Average baseline: all metrics = 0.5

# Initialize polar plot
fig, ax = plt.subplots(figsize=(8, 8), subplot_kw=dict(polar=True), dpi=160)

# Define colors and transparency levels
color_mean = "#2E8B57"        # SeaGreen for the mean line/fill
color_fill = "#2E8B57"
alpha_main = 0.99             # Almost opaque mean fill
alpha_std = 0.25              # Light transparency for std deviation band
label_color = "#2F4F4F"       # DarkSlateGray (not used here but ready)

color_best = "#1f77b4"        # Blue for best case line
color_worst = "#d62728"       # Red for worst case markers
color_avg_case = "#ff7f0e"    # Orange for average case line

# Plot mean metric line and fill area
ax.plot(angles, metrics, color=color_mean, linewidth=2.5, label="Mean")
ax.fill(angles, metrics, color=color_fill, alpha=alpha_main)

# Shade ±1 standard deviation around the mean
ax.fill_between(angles, lower, upper, color=color_fill, alpha=alpha_std, label="± 1 Std Dev")

# Plot best case as a dashed blue line
ax.plot(angles, best_case, color=color_best, linewidth=2, linestyle='--', label="Best Case (1.0)")

# Plot worst case as red dots at zero radius on each axis
ax.scatter(angles[:-1], worst_case[:-1], color=color_worst, s=100, marker='o', label="Worst Case (0.0)")

# Plot average baseline as a dashed orange line
ax.plot(angles, avg_case, color=color_avg_case, linewidth=2, linestyle='--', label="Average Case (0.5)")

# Configure axis labels with large, bold font
ax.set_thetagrids(np.degrees(angles[:-1]), labels, fontsize=30, fontweight='semibold', color="black")

# Set radial ticks and labels from 0 to 1, spaced evenly
radial_ticks = np.linspace(0.0, 1.0, 6)  # [0.0, 0.2, ..., 1.0]
ax.set_yticks(radial_ticks)
ax.set_yticklabels([f"{x:.1f}" for x in radial_ticks], fontsize=20, color='black')
ax.yaxis.grid(True, color='black', linestyle='--', alpha=0.7, linewidth=1)
ax.set_rlim(0, 1.0)  # Set radial axis limits

# Adjust plot rotation and direction for better readability
ax.set_theta_offset(np.pi / 2)  # Start from top
ax.set_theta_direction(-1)      # Clockwise ordering

# Hide the circular frame and set background color
ax.spines["polar"].set_visible(False)
ax.set_facecolor("white")

# Annotate mean metric values near each axis for clarity
for angle, value in zip(angles[:-1], metrics[:-1]):
    # Align text depending on angle quadrant to avoid overlap
    if 0 <= angle <= np.pi / 2:
        ha, va, dy = 'left', 'bottom', 0.04
    elif np.pi / 2 < angle <= np.pi:
        ha, va, dy = 'right', 'bottom', 0.04
    elif np.pi < angle <= 3*np.pi/2:
        ha, va, dy = 'right', 'top', -0.04
    else:
        ha, va, dy = 'left', 'top', -0.04

    ax.text(angle, value + dy, f"{value:.2f}",
            ha=ha, va=va, fontsize=20, fontweight='medium', color='black',
            bbox=dict(boxstyle='round,pad=0.25', facecolor='white', edgecolor=color_mean, linewidth=0.8, alpha=0.85))

# Add legend outside the plot to the right
ax.legend(loc='upper right', bbox_to_anchor=(1.4, 1.2), fontsize=14)

# Optimize layout to prevent clipping
plt.tight_layout()
plt.show()



In [ ]:
import os
import pandas as pd
import numpy as np
from sklearn.manifold import MDS
import matplotlib.pyplot as plt
import re

# --- CONFIG ---
shap_dir = "/Users/tuanadurmayuksel/Desktop/Internship_code/hbn-dataset-special-data/HBN /Final/montage/disorder_pair_results/shap_values/Shap"
shap_files = [f for f in os.listdir(shap_dir) if f.startswith("shap_") and f.endswith(".csv")]

# --- HELPER: extract disorder pair name ---
def extract_pair_name(filename):
    match = re.match(r"shap_(.*?)_vs_(.*?)_fold\d+\.csv", filename)
    if match:
        return f"{match.group(1).replace('_', ' ')} vs {match.group(2).replace('_', ' ')}"
    return None

# --- STEP 1: Aggregate SHAP values per disorder pair ---
pair_shap_dict = {}

for file in shap_files:
    filepath = os.path.join(shap_dir, file)
    pair_name = extract_pair_name(file)
    
    if pair_name is None:
        continue  # skip files that don't match pattern
    
    df = pd.read_csv(filepath)
    shap_vals = df.drop(columns=['label'])
    mean_abs_shap = shap_vals.abs().mean(axis=0)
    
    if pair_name not in pair_shap_dict:
        pair_shap_dict[pair_name] = []
    pair_shap_dict[pair_name].append(mean_abs_shap)

# --- STEP 1.5: Collect all features across all pairs ---
all_features = set()
for shap_list in pair_shap_dict.values():
    for mean_abs_shap in shap_list:
        all_features.update(mean_abs_shap.index)

all_features = sorted(list(all_features))

# --- STEP 2: Final SHAP matrix per disorder pair (mean across folds, aligned features) ---
pair_names = []
shap_matrix = []

for pair, shap_list in pair_shap_dict.items():
    pair_names.append(pair)
    
    # Reindex each fold's mean vector to include all features, fill missing with 0
    reindexed = [s.reindex(all_features, fill_value=0) for s in shap_list]
    
    # Concatenate along columns and average across folds
    mean_vector = pd.concat(reindexed, axis=1).mean(axis=1)
    shap_matrix.append(mean_vector.values)

shap_matrix = np.vstack(shap_matrix)

# --- STEP 3: Apply MDS ---
mds = MDS(n_components=2, random_state=42)
coords = mds.fit_transform(shap_matrix)
coords_scaled = coords * 2.5  # optional scaling

# --- STEP 4: Define disorder short name map ---
disorder_name_map = {
    'ASD': 'ASD', 'ADHD-Combined Type': 'ADHD-C', 'Healthy controls': 'Healthy',
    'Anxiety Disorders': 'Anxiety', 'SLD (Reading)': 'SLD-Read', 'SLD (Mathematics)': 'SLD-Math',
    'SLD (Written Expression)': 'SLD-Write', 'OCD': 'OCD', 'OCD-Other': 'OCD-Other',
    'Communication Disorders': 'CommDis', 'Intellectual Disability': 'ID',
    'Depressive Disorders': 'Depression', 'Trauma and Stressor Related Disorders': 'Trauma-Stress',
    'Oppositional Defiant Disorder': 'ODD', 
    'Disruptive, Impulse Control and Conduct Disorders-Other': 'Impulse',
    'Tic Disorders': 'Tic', 'Elimination Disorders': 'Elimination',
    'ADHD-Hyperactive/Impulsive Type': 'ADHD-H', 'ADHD-Inattentive Type': 'ADHD-I'
}

# --- STEP 5: Plot MDS with short disorder names ---
fig1, ax1 = plt.subplots(figsize=(12, 8), dpi=100)
ax1.scatter(coords_scaled[:, 0], coords_scaled[:, 1], c='lightgray', s=100, edgecolors='black', linewidth=0.5)

def get_short_name(full_name):
    return disorder_name_map.get(full_name, full_name)

# offset = 0.01
# offset1 = 0.005

for i, (x, y) in enumerate(coords_scaled):
    disorder1, disorder2 = pair_names[i].split(" vs ")
    short_label = f"{get_short_name(disorder1)} vs {get_short_name(disorder2)}"
    ax1.text(x+0.005 , y+0.02 , short_label, fontsize=13, fontweight='bold', ha='center', va='center', color='black')

ax1.set_xlabel("MDS Dimension 1", fontsize=16)
ax1.set_ylabel("MDS Dimension 2", fontsize=16)
ax1.grid(True, linestyle='--', alpha=0.3)
ax1.tick_params(labelsize=20)
ax1.spines[['top', 'right']].set_visible(False)

plt.tight_layout()
plt.show()


In [ ]:
# --- STEP 5: Plot MDS with bigger dots and numbers, better axis labels, and numbered legend below ---

fig1, ax1 = plt.subplots(figsize=(12, 11), dpi=100)  # extra height for labels below

dot_size = 1500  # bigger dots
number_fontsize = 30  # bigger numbers on dots
legend_fontsize = 20  # bigger labels below

ax1.scatter(coords_scaled[:, 0], coords_scaled[:, 1], c='lightgrey', s=dot_size, edgecolors='black', linewidth=0.7)

# Number each dot with bigger, bold text centered on the dots
for i, (x, y) in enumerate(coords_scaled):
    ax1.text(x, y, str(i+1), fontsize=number_fontsize, fontweight='bold', ha='center', va='center', color='black')

# Better axis labels
ax1.set_xlabel("MDS Dimension 1 ", fontsize=30, fontweight='bold')
ax1.set_ylabel("MDS Dimension 2 ", fontsize=30, fontweight='bold')

ax1.grid(True, linestyle='--', alpha=0.3)
ax1.tick_params(labelsize=14)
ax1.spines[['top', 'right']].set_visible(False)

# Prepare numbered disorder pair list with bigger font, separated by newlines
numbered_labels = [f"{i+1}. {get_short_name(pair.split(' vs ')[0])} vs {get_short_name(pair.split(' vs ')[1])}" 
                   for i, pair in enumerate(pair_names)]

# Adjust bottom margin to fit labels
plt.subplots_adjust(bottom=0.35)

# Place labels below the plot, centered and spaced nicely
fig1.text(0.5, 0.001, "\n".join(numbered_labels), va='top', fontsize=legend_fontsize)

plt.tight_layout()
plt.show()


In [ ]:
import pandas as pd
import numpy as np
import mne
import matplotlib.pyplot as plt
from matplotlib import cm

# --- Configuration parameters ---
threshold = 0.01                      # Minimum normalized SHAP value to visualize
montage_type = 'GSN-HydroCel-128'    # EEG sensor layout to use for topomap plotting
figsize = (2, 2)                     # Size of each individual topomap subplot
font_size = 8                        # Font size for subplot titles and labels
max_cols = 6                        # Max number of columns in the subplot grid
max_rows = 5                        # Max number of rows in the subplot grid

# Map feature names to human-readable labels and filter for positive SHAP values only
df_mapped_result = map_feature_descriptions(df_result, top_n=1000)
df = df_mapped_result.copy()
df = df[df['Avg_SHAP'] > 0]          # Keep only features with positive average SHAP

# Prepare EEG channel info and 2D sensor positions for topomap plotting
montage = mne.channels.make_standard_montage(montage_type)
ch_names = montage.ch_names
info = mne.create_info(ch_names=ch_names, sfreq=256, ch_types='eeg')
info.set_montage(montage)
pos = np.array([info['chs'][i]['loc'][:2] for i in range(len(info['chs']))])  # 2D sensor coords

# Group data by Frequency and Biomarker to plot separate topomaps for each combination
combinations = list(df.groupby(['Frequency', 'Biomarker']))
combinations = combinations[:max_cols * max_rows]  # Limit to max plots that fit grid

n_plots = len(combinations)
n_cols = min(max_cols, n_plots)                 # Calculate grid columns based on data & max allowed
n_rows = min(max_rows, int(np.ceil(n_plots / n_cols)))  # Calculate rows to fit all plots

# Compute overall figure size based on grid layout and individual subplot size
fig_width = figsize[0] * n_cols
fig_height = figsize[1] * n_rows

# Create a figure with a grid of subplots (polar topomaps)
fig, axes = plt.subplots(n_rows, n_cols, figsize=(fig_width, fig_height))
axes = axes.flatten() if n_plots > 1 else [axes]  # Flatten axes array for easy iteration

cmap = plt.cm.bwr  # Blue-white-red colormap for visualizing positive/negative values (here just positive)

# Loop over each Frequency-Biomarker group to compute and plot topomap of SHAP importance
for i, ((freq, biom), group) in enumerate(combinations):
    # Initialize dictionary with 0 importance for all EEG channels
    importance_map = {ch: 0 for ch in ch_names}
    
    # Aggregate Avg_SHAP values per channel/source for the current group
    for _, row in group.iterrows():
        if row['Source'] in importance_map:
            importance_map[row['Source']] += row['Avg_SHAP']

    # Normalize importance values to [0,1] for coloring on the topomap
    values = np.array(list(importance_map.values()))
    norm_values = (values - values.min()) / (values.max() - values.min() + 1e-8)

    # Apply threshold: values below threshold are set to zero (not visualized)
    norm_map = {ch: (val if val > threshold else 0) for ch, val in zip(ch_names, norm_values)}
    data = np.array([norm_map[ch] for ch in ch_names])

    # Plot the topomap on the corresponding subplot axis
    ax = axes[i]
    im, _ = mne.viz.plot_topomap(data, pos, axes=ax, show=False, contours=0, cmap=cmap)
    ax.set_title(f"{freq}\n{biom}", fontsize=font_size)  # Title with frequency and biomarker
    ax.axis('off')  # Hide axis ticks

# Turn off axes for any unused subplot slots
for j in range(i + 1, len(axes)):
    axes[j].axis('off')

# Add a vertical colorbar to the right of the subplots for importance scale
cbar_ax = fig.add_axes([0.93, 0.2, 0.015, 0.6])
fig.colorbar(im, cax=cbar_ax, label='Normalized Importance')

# Add an overall title for the figure
plt.suptitle("EEG SHAP Importance by Frequency & Biomarker", fontsize=font_size + 2)

# Adjust layout to fit subplot titles and colorbar nicely
plt.tight_layout(rect=[0, 0, 0.92, 1])
plt.subplots_adjust(top=0.9)

# Display the complete figure with all topomap subplots
plt.show()


In [ ]:
# Decode all features, not just top 20
mapped_all = [parse_feature_name(feat) for feat in df_result['Feature']]
shap_to_original = pd.DataFrame(mapped_all)
shap_to_original['importance'] = df_result['Avg_SHAP']  # Match with SHAP values


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Aggregate mean importance values by Biomarker and Frequency, reshape to wide format
heatmap_data = shap_to_original.groupby(['Biomarker', 'Frequency'])['importance'].mean().unstack()

# Check if all values are zero to avoid meaningless plot
if np.max(heatmap_data.values) == 0:
    print("All coefficient values are zero! Cannot plot meaningful heatmap.")
else:
    # Normalize data by dividing by max value for color scaling between 0 and 1
    heatmap_data_norm = heatmap_data / np.max(heatmap_data.values)

    # Clean column names by stripping whitespace and rename specific frequency bands
    heatmap_data_norm.columns = heatmap_data_norm.columns.str.strip()
    heatmap_data_norm = heatmap_data_norm.rename(columns={
        'RelativePower': 'RP',
        'AbsolutePower': 'AP'
    })

    # Clean index (Biomarker) names similarly and rename if needed
    heatmap_data_norm.index = heatmap_data_norm.index.str.strip()
    heatmap_data_norm = heatmap_data_norm.rename(index={
        'RelativePower': 'RP',
        'AbsolutePower': 'AP'
        # Add more biomarker renames here if necessary
    })

    # Create the heatmap figure with specified size
    plt.figure(figsize=(12, 6))

    # Plot heatmap with color mapping, annotations, and styling
    ax = sns.heatmap(
        heatmap_data_norm,
        cmap="YlGnBu",
        cbar_kws={'aspect': 35},     # Aspect ratio for colorbar
        annot=True,                  # Show values on heatmap
        fmt=".2f",                  # Format annotation to 2 decimal places
        annot_kws={"size": 20},     # Font size for annotations
        linecolor='black',           # Grid line color between cells
        linewidth=1                  # Grid line width
    )

    # Set axis labels with font sizes and rotations
    plt.xlabel("Frequency Band", fontsize=18)
    plt.ylabel("", fontsize=18)  # Leave y-axis label blank for custom label

    plt.xticks(fontsize=16, rotation=25, fontweight='bold')
    plt.yticks(fontsize=16, rotation=0, fontweight='bold')

    # Remove left spine for cleaner look
    ax.spines['left'].set_visible(False)

    # Add custom rotated label "Biomarker" to y-axis with positioning
    ax.yaxis.set_label_coords(-0.1, 0.5)
    ax.text(
        x=-0.10, y=0.40, s="Biomarker",
        fontsize=18, rotation=90,
        ha='center', va='bottom',
        transform=ax.transAxes
    )

    # Customize colorbar ticks and label font size
    cbar = ax.collections[0].colorbar
    cbar.ax.tick_params(labelsize=20)
    cbar.set_label('Normalized SHAP Importance', fontsize=20)

    # Adjust layout to avoid overlap
    plt.tight_layout()

    # Display the heatmap plot
    plt.show()


